In [189]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import datetime
import math
import itertools
from scipy import optimize
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sys
sys.path.append("..")
from RiskPackage.CalculateReturn import return_calculate
from RiskPackage.RiskMetrics import RiskMetrics
from scipy import linalg

# Problem 1
- Current Stock Price $165
- Strike Price $165
- Current Date 03/13/2022
- Options Expiration Date 04/15/2022
- Risk Free Rate of 4.25%
- Continuously Compounding Coupon of 0.53%
- implied volatility 20%

Implement the closed form greeks for GBSM. Implement a finite difference derivative calculation.
Compare the values between the two methods for both a call and a put.

Implement the binomial tree valuation for American options with and without discrete dividends. 

Assume the stock above:
- Pays dividend on 4/11/2022 of $0.88

Calculate the value of the call and the put. Calculate the Greeks of each.

What is the sensitivity of the put and call to a change in the dividend amount?


In [4]:
def gbsm(s,strike,ttm,vol,rf,c,call=True):
    '''
    Generalize Black Scholes Merton
    rf = c       -- Black Scholes 1973
    c = rf - q   -- Merton 1973 stock model where q is the continous dividend yield
    c = 0        -- Black 1976 futures option model
    c,r = 0      -- Asay 1982 margined futures option model
    c = rf - rff -- Garman and Kohlhagen 1983 currency option model where rff is the risk free rate of the foreign currency

    Option valuation via BSM closed formula
    European Style.  Assumed LogNormal Prices
    s - Underlying Price
    strike - Strike Price
    ttm - time to maturity
    rf - Risk free rate
    vol - Yearly Volatility
    c - Cost of Carry
    call - Call valuation if set True
    '''
    d1=(np.log(s/strike)+(c+vol**2/2)*ttm)/vol/np.sqrt(ttm)
    d2=d1-vol*np.sqrt(ttm)
    if call:
        return s*np.exp((c-rf)*ttm)*norm.cdf(d1)-strike*np.exp(-rf*ttm)*norm.cdf(d2)
    else:
        return strike*np.exp(-rf*ttm)*norm.cdf(-d2)-s*np.exp((c-rf)*ttm)*norm.cdf(-d1)

In [5]:
def greeks_closed_form(s,strike,ttm,vol,rf,c,call=True):
    '''Closed from for greeks calculation from Generalize Black Scholes Merton

    Generalize Black Scholes Merton:
    rf = c       -- Black Scholes 1973
    c = rf - q   -- Merton 1973 stock model where q is the continous dividend yield
    c = 0        -- Black 1976 futures option model
    c,r = 0      -- Asay 1982 margined futures option model
    c = rf - rff -- Garman and Kohlhagen 1983 currency option model where rff is the risk free rate of the foreign currency

    Option valuation via BSM closed formula
    European Style.  Assumed LogNormal Prices
    s - Underlying Price
    strike - Strike Price
    ttm - time to maturity
    rf - Risk free rate
    vol - Yearly Volatility
    c - Cost of Carry
    call - Call valuation if set True
    '''
    d1=(np.log(s/strike)+(c+vol**2/2)*ttm)/vol/np.sqrt(ttm)
    d2=d1-vol*np.sqrt(ttm)
    optionType=['Call'] if call else ['Put']
    ans=pd.DataFrame(index=optionType,columns=['Detla','Gamma','Vega','Theta','Rho','Carry Rho'])
    if call:
        ans['Detla'] = np.exp((c-rf)*ttm)*norm.cdf(d1,loc=0,scale=1)
        ans['Theta'] = -s*np.exp((c-rf)*ttm)*norm.pdf(d1,loc=0,scale=1)*vol/(2*np.sqrt(ttm))-(c-rf)*s*np.exp((c-rf)*ttm)*norm.cdf(d1,loc=0,scale=1)-rf*strike*np.exp(-rf*ttm)*norm.cdf(d2,loc=0,scale=1)
        # ans['Rho'] = ttm*strike*np.exp(-rf*ttm)*norm.cdf(d2,loc=0,scale=1) - s*ttm*np.exp((c-rf)*ttm)*norm.cdf(d1,loc=0,scale=1)
        ans['Rho'] = ttm*strike*np.exp(-rf*ttm)*norm.cdf(d2,loc=0,scale=1)

        ans['Carry Rho'] = ttm*s*np.exp((c-rf)*ttm)*norm.cdf(d1,loc=0,scale=1)
    else:
        ans['Detla'] = np.exp((c-rf)*ttm)*(norm.cdf(d1,loc=0,scale=1)-1)
        ans['Theta'] = -s*np.exp((c-rf)*ttm)*norm.pdf(d1,loc=0,scale=1)*vol/(2*np.sqrt(ttm))+(c-rf)*s*np.exp((c-rf)*ttm)*norm.cdf(-d1,loc=0,scale=1)+rf*strike*np.exp(-rf*ttm)*norm.cdf(-d2,loc=0,scale=1)
        ans['Rho'] = -ttm*strike*np.exp(-rf*ttm)*norm.cdf(-d2,loc=0,scale=1)
        # ans['Rho'] = -ttm*strike*np.exp(-rf*ttm)*norm.cdf(-d2,loc=0,scale=1)+ttm*s*norm.cdf(-d1,loc=0,scale=1)*exp((b-rf)*ttm)
        ans['Carry Rho'] = -ttm*s*np.exp((c-rf)*ttm)*norm.cdf(-d1,loc=0,scale=1)
    ans['Gamma'] = norm.pdf(d1,loc=0,scale=1)*np.exp((c-rf)*ttm)/(s*vol*np.sqrt(ttm))
    ans['Vega'] = s*np.exp((c-rf)*ttm)*norm.pdf(d1,loc=0,scale=1)*np.sqrt(ttm)

    return ans

In [6]:
def binomial_tree_gbsm_american(s,strike,ttm,vol,rf,c,N=200,call=True):
    '''
    Generalize Black Scholes Merton
    rf = c       -- Black Scholes 1973
    c = rf - q   -- Merton 1973 stock model where q is the continous dividend yield
    c = 0        -- Black 1976 futures option model
    c,r = 0      -- Asay 1982 margined futures option model
    c = rf - rff -- Garman and Kohlhagen 1983 currency option model where rff is the risk free rate of the foreign currency

    Option valuation via Binomial Trees
    European Style.  Assumed LogNormal Prices
    s - Underlying Price
    strike - Strike Price
    t - days until maturity
    rf - Risk free rate
    vol - Yearly Volatility
    c - Cost of Carry
    tradingDayYear - trading days in a year
    N - Steps of Binomial Tree
    call - Call valuation if set True
    '''
    delta_t=ttm/N
    # price multiplier in the (positive, negtive) case
    u=np.exp(vol*np.sqrt(delta_t)) 
    d=np.exp(-vol*np.sqrt(delta_t))
    # probability of up/down move in price
    pu=(np.exp(c*delta_t)-d)/(u-d)
    pd=1-pu
    # discount factor
    df=np.exp(-rf*delta_t)
    # call or put
    optionType=1 if call else -1

    # American option
    nNodeFunction = lambda x: (x+1)*(x+2)//2 # Calculate the number of all the nodes
    # j: layer(step) of nodes (python start by 0)
    #i: the i-th node in the j-th step (python start by 0)
    idxFunction = lambda i,j: nNodeFunction(j-1)+i
    nNodes=nNodeFunction(N) # Number of nodes
    
    optionValues = np.empty(nNodes,dtype=float) # An array of all the nodes

    for j in range(N,-1,-1): 
        for i in range(j,-1,-1):
            idx = idxFunction(i,j) 
            price = s*u**i*d**(j-i) # i represt how many times price go up
            optionValues[idx]=max(0,optionType*(price-strike))
            if j < N:
                optionValues[idx]=max(optionValues[idx],df*(pu*optionValues[idxFunction(i+1,j+1)] + pd*optionValues[idxFunction(i,j+1)]))
    return optionValues[0]

In [7]:
def binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N=200,call=True,divAmts=None,divTimes=None):
    '''
    Generalize Black Scholes Merton
    rf = c       -- Black Scholes 1973
    c = rf - q   -- Merton 1973 stock model where q is the continous dividend yield
    c = 0        -- Black 1976 futures option model
    c,r = 0      -- Asay 1982 margined futures option model
    c = rf - rff -- Garman and Kohlhagen 1983 currency option model where rff is the risk free rate of the foreign currency

    Option valuation via Binomial Trees
    European Style.  Assumed LogNormal Prices
    s - Underlying Price
    strike - Strike Price
    t - days until maturity
    rf - Risk free rate
    vol - Yearly Volatility
    c - Cost of Carry
    tradingDayYear - trading days in a year
    N - Steps of Binomial Tree
    call - Call valuation if set True

    divAmts - Array of dividend amounts
    divTimes - Array of dividend times
    '''
    #if there are no dividends or the first dividend is outside out grid, return the standard bt_american value
    divAmts=np.array(divAmts)
    divTimes=np.array(divTimes).astype(np.int32)
    if  divAmts.size == 0 or divTimes.size == 0 or divTimes[0]>N:
        return binomial_tree_gbsm_american(s,strike,ttm,vol,rf,c,N,call)
        
    delta_t=ttm/N
    # price multiplier in the (positive, negtive) case
    u=np.exp(vol*np.sqrt(delta_t)) 
    d=np.exp(-vol*np.sqrt(delta_t))
    # probability of up/down move in price
    pu=(np.exp(c*delta_t)-d)/(u-d)
    pd=1-pu
    # discount factor
    df=np.exp(-rf*delta_t)
    # call or put
    optionType=1 if call else -1

    # American option
    nNodeFunction = lambda x: (x+1)*(x+2)//2 # Calculate the number of all the nodes
    # j: layer(step) of nodes (python start by 0)
    #i: the i-th node in the j-th step (python start by 0)
    idxFunction = lambda i,j: nNodeFunction(j-1)+i
    nNodes=nNodeFunction(divTimes[0]) # Number of nodes

    optionValues = np.empty(nNodes,dtype=float) # An array of all the nodes

    for j in range(divTimes[0],-1,-1): 
        for i in range(j,-1,-1):
            idx = idxFunction(i,j) 
            price = s*u**i*d**(j-i) # i represt how many times price go up
            if j < divTimes[0]:
                # times before the dividend working backward induction
                optionValues[idx]=max(0,optionType*(price-strike))
                optionValues[idx]=max(optionValues[idx],df*(pu*optionValues[idxFunction(i+1,j+1)] + pd*optionValues[idxFunction(i,j+1)]))
            else:
                # time of the dividend
               valNoExercise = binomial_tree_gbsm_american_div(price-divAmts[0],strike,ttm-divTimes[0]*delta_t,vol,rf,c,N-divTimes[0],call,divAmts[1:],divTimes[1:]-divTimes[0])
               valExercise =  max(0,optionType*(price-strike))
               optionValues[idx] = max(valNoExercise,valExercise)
    return optionValues[0]

In [8]:
def greeks_gbsm_finite_diff(s,strike,ttm,vol,rf,c,call=True):
    '''Greeks for European option via Generalize Black Scholes Merton and finite difference'''
    optionType=['Call'] if call else ['Put']
    s_chg = 0.01 # amount of price change 
    vol_chg = 0.01 # amount of volatility change
    t_chg = 0.01 # amount of ttm change
    rf_chg = 0.01 # amount of rf change
    c_chg = 0.01 # amount of carry of cost change

    ans=pd.DataFrame(index=optionType,columns=['Detla','Gamma','Vega','Theta','Rho','Carry Rho'])

    price_price_m_s_chg=gbsm(s-s_chg,strike,ttm,vol,rf,c,call) # price-s_chg
    price_price_p_s_chg=gbsm(s+s_chg,strike,ttm,vol,rf,c,call) # price+s_chg
    price=gbsm(s,strike,ttm,vol,rf,c,call)

    ans['Detla']=(price_price_p_s_chg-price_price_m_s_chg)/(2*s_chg)
    
    ans['Gamma']=(price_price_p_s_chg+price_price_m_s_chg-2*price)/s_chg**2

    ans['Vega']=(gbsm(s,strike,ttm,vol+vol_chg,rf,c,call)-gbsm(s,strike,ttm,vol-vol_chg,rf,c,call))/(2*vol_chg)

    ans['Theta']=-(gbsm(s,strike,ttm+t_chg,vol,rf,c,call)-gbsm(s,strike,ttm-t_chg,vol,rf,c,call))/(2*t_chg)

    ans['Rho']=(gbsm(s,strike,ttm,vol,rf+rf_chg,c+rf_chg,call)-gbsm(s,strike,ttm,vol,rf-rf_chg,c-rf_chg,call))/(2*rf_chg) # c should also be changed since c=rf-q 

    ans['Carry Rho']=(gbsm(s,strike,ttm,vol,rf,c+c_chg,call)-gbsm(s,strike,ttm,vol,rf,c-c_chg,call))/(2*c_chg)

    return ans
    

In [9]:
def greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,c,N=200,call=True,divAmts=None,divTimes=None):
    '''Greeks for American option via binomial tree and finite difference'''
    optionType=['Call'] if call else ['Put']
    s_chg = 0.2 # amount of price change 
    vol_chg = 0.01 # amount of volatility change
    t_chg = 0.01 # amount of ttm change
    rf_chg = 0.01 # amount of rf change
    c_chg = 0.01 # amount of carry of cost change
    divAmts_chg=0.01 # amount of dividend amount change

    ans=pd.DataFrame(index=optionType,columns=['Detla','Gamma','Vega','Theta','Rho','Carry Rho','Sensitivity to Dividend Amount'])

    price_price_m_s_chg=binomial_tree_gbsm_american_div(s-s_chg,strike,ttm,vol,rf,c,N,call,divAmts,divTimes) # price-s_chg
    price_price_p_s_chg=binomial_tree_gbsm_american_div(s+s_chg,strike,ttm,vol,rf,c,N,call,divAmts,divTimes) # price+s_chg
    price=binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,call,divAmts,divTimes) 

    ans['Detla']=(price_price_p_s_chg-price_price_m_s_chg)/(2*s_chg)
    
    ans['Gamma']=(price_price_p_s_chg+price_price_m_s_chg-2*price)/s_chg**2

    ans['Vega']=(binomial_tree_gbsm_american_div(s,strike,ttm,vol+vol_chg,rf,c,N,call,divAmts,divTimes)-binomial_tree_gbsm_american_div(s,strike,ttm,vol-vol_chg,rf,c,N,call,divAmts,divTimes))/(2*vol_chg)

    ans['Theta']=-(binomial_tree_gbsm_american_div(s,strike,ttm+t_chg,vol,rf,c,N,call,divAmts,divTimes) - binomial_tree_gbsm_american_div(s,strike,ttm-t_chg,vol,rf,c,N,call,divAmts,divTimes) )/(2*t_chg)

    ans['Rho']=(binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf+rf_chg,c+rf_chg,N,call,divAmts,divTimes) - binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf-rf_chg,c-rf_chg,N,call,divAmts,divTimes) )/(2*rf_chg) # c should also be changed since c=rf-q

    ans['Carry Rho']=(binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c+c_chg,N,call,divAmts,divTimes)  - binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c-c_chg,N,call,divAmts,divTimes) )/(2*c_chg)

    if divAmts and divTimes:
        divAmts = np.array(divAmts)
        divTimes = np.array(divTimes)
        ans['Sensitivity to Dividend Amount'] = (binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,call,divAmts+divAmts_chg,divTimes)  - binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,call,divAmts-divAmts_chg,divTimes) )/(2*divAmts_chg)


    return ans

In [10]:
# Initialization
CurrentDate=datetime.datetime.strptime('03/13/2022','%m/%d/%Y')
ExpirationDate=datetime.datetime.strptime('04/15/2022','%m/%d/%Y')
Dividend_date=datetime.datetime.strptime('04/11/2022','%m/%d/%Y')

t=(ExpirationDate-CurrentDate).days
tradingDayYear=365
divTimes=(Dividend_date-CurrentDate).days

s=165
strike=165
rf=0.0425
q=0.0053
c=rf-q
ttm=t/tradingDayYear
vol=0.2

N=200
divTimes=[divTimes/t*N]
divAmts=[0.88]

In [11]:
# Closed Form of Greeks Vs. Finite Difference derivative calculation
call_greeks_closed_form=greeks_closed_form(s,strike,ttm,vol,rf,c,call=True)
put_greeks_closed_form=greeks_closed_form(s,strike,ttm,vol,rf,c,call=False)
call_greeks_gbsm_finite_diff=greeks_gbsm_finite_diff(s,strike,ttm,vol,rf,c,call=True)
put_greeks_gbsm_finite_diff=greeks_gbsm_finite_diff(s,strike,ttm,vol,rf,c,call=False)

# European
pd.concat([call_greeks_closed_form,put_greeks_closed_form,call_greeks_gbsm_finite_diff,put_greeks_gbsm_finite_diff]).set_index([['Closed Form','Closed Form','Finite Difference','Finite Difference'],['Call','Put','Call','Put']])

Detla     Gamma       Vega      Theta       Rho  \
Closed Form       Call  0.534009  0.040038  19.710180 -24.898522  7.583586   
                  Put  -0.465512  0.040038  19.710180 -18.786997 -7.277011   
Finite Difference Call  0.534009  0.040038  19.710095 -24.932205  7.583554   
                  Put  -0.465512  0.040038  19.710095 -18.820680 -7.277045   

                        Carry Rho  
Closed Form       Call   7.966246  
                  Put   -6.944416  
Finite Difference Call   7.966254  
                  Put   -6.944409

In [22]:
# Valuation 

# Continuously Compounding Coupon
# without dividend
call_coupon_no_dividend = binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,True,[],[])
put_coupon_no_dividend = binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,False,[],[])
# With dividend
call_coupon_dividend = binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,True,divAmts,divTimes)
put_coupon_dividend = binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,False,divAmts,divTimes)

# No Coupon
# without dividend
call_no_dividend = binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,rf,N,True,[],[])
put_no_dividend = binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,rf,N,False,[],[])
# With dividend
call_dividend = binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,rf,N,True,divAmts,divTimes)
put_dividend = binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,rf,N,False,divAmts,divTimes)


# Greeks derived by binomial tree and finite difference

# Continuously Compounding Coupon
# without dividend (Continuously Compounding Coupon of 0.53%)
call_greeks_coupon_no_dividend=greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,c,N,True,[],[])
put_greeks_coupon_no_dividend=greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,c,N,False,[],[])
# With dividend (Continuously Compounding Coupon of 0.53% + Discrete dividends)
call_greeks_coupon_dividend=greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,c,N,True,divAmts,divTimes)
put_greeks_coupon_dividend=greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,c,N,False,divAmts,divTimes)

# No coupon
# without dividend (No coupon)
call_greeks_no_dividend=greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,rf,N,True,[],[])
put_greeks_no_dividend=greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,rf,N,False,[],[])
# With dividend (No coupon + Discrete dividends)
call_greeks_dividend=greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,rf,N,True,divAmts,divTimes)
put_greeks_dividend=greeks_binomial_tree_finite_diff(s,strike,ttm,vol,rf,rf,N,False,divAmts,divTimes)

index=list(itertools.product(['Continuously Compounding Coupon','No Coupon'],['Without Discrete Dividend','With Discrete Dividend'],['Call','Put']))

Value=pd.DataFrame([call_coupon_no_dividend,put_coupon_no_dividend,call_coupon_dividend,put_coupon_dividend,call_no_dividend,put_no_dividend,call_dividend,put_dividend],columns=['Valuation']).set_index([index])

Greeks = pd.concat([call_greeks_coupon_no_dividend,put_greeks_coupon_no_dividend,call_greeks_coupon_dividend,put_greeks_coupon_dividend,call_greeks_no_dividend,put_greeks_no_dividend,call_greeks_dividend,put_greeks_dividend]).set_index([index])

ValueGreeks = pd.concat([Value,Greeks],axis=1)
ValueGreeks


Valuation  \
Continuously Compounding Coupon Without Discrete Dividend Call   4.227506   
                                                          Put    3.714324   
                                With Discrete Dividend    Call   4.074976   
                                                          Put    4.147062   
No Coupon                       Without Discrete Dividend Call   4.269859   
                                                          Put    3.684138   
                                With Discrete Dividend    Call   4.112836   
                                                          Put    4.110535   

                                                                   Detla  \
Continuously Compounding Coupon Without Discrete Dividend Call  0.533966   
                                                          Put  -0.473668   
                                With Discrete Dividend    Call  0.530507   
                                                          Put  -0.495856   
No Coupon                       Without Discrete Dividend Call  0.537384   
                                                          Put  -0.471987   
                                With Discrete Dividend    Call  0.532904   
                                                          Put  -0.493280   

                                                                   Gamma  \
Continuously Compounding Coupon Without Discrete Dividend Call  0.280568   
                                                          Put   0.246837   
                                With Discrete Dividend    Call  0.031063   
                                                          Put   0.002387   
No Coupon                       Without Discrete Dividend Call  0.280501   
                                                          Put   0.242952   
                                With Discrete Dividend    Call  0.021277   
                                                          Put   0.003252   

                                                                     Vega  \
Continuously Compounding Coupon Without Discrete Dividend Call  19.685473   
                                                          Put   19.655211   
                                With Discrete Dividend    Call  19.576769   
                                                          Put   19.815527   
No Coupon                       Without Discrete Dividend Call  19.680769   
                                                          Put   19.642167   
                                With Discrete Dividend    Call  19.574566   
                                                          Put   19.824035   

                                                                    Theta  \
Continuously Compounding Coupon Without Discrete Dividend Call -24.904878   
                                                          Put  -19.299822   
                                With Discrete Dividend    Call -24.464277   
                                                          Put  -18.948359   
No Coupon                       Without Discrete Dividend Call -25.388799   
                                                          Put  -18.988711   
                                With Discrete Dividend    Call -24.897100   
                                                          Put  -18.573133   

                                                                     Rho  \
Continuously Compounding Coupon Without Discrete Dividend Call  7.583365   
                                                          Put  -5.980132   
                                With Discrete Dividend    Call  6.797344   
                                                          Put  -7.248928   
No Coupon                       Without Discrete Dividend Call  7.630524   
                                                          Put  -5.896065   
                                With Discrete Dividend    Call  6.834995   
    

# Problem 2
Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and
assuming :
- American Options
- Current Date 03/03/2023
- Current AAPL price is 151.03
- Risk Free Rate of 4.25%
- Dividend Payment of $1.00 on 3/15/2023

Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above). Calculate Mean, VaR and ES.

Calculate VaR and ES using Delta-Normal.

Present all VaR and ES values a $ loss, not percentages.

Compare these results to last week’s results.

In [46]:
# Calculate the implied volatility of each option
df=pd.read_csv('problem2.csv')

def findImpliedVol_plus(data):
    '''Remind: please set the initial value before use'''

    if data['Type']!='Option': # if the security is not option return nan
        return np.nan

    tradingDayYear=365

    CurrentDate=datetime.datetime.strptime('03/03/2023','%m/%d/%Y')
    ExpirationDate=data['ExpirationDate']
    Dividend_date=datetime.datetime.strptime('03/15/2023','%m/%d/%Y')
    ExpirationDate=datetime.datetime.strptime(ExpirationDate,'%m/%d/%Y')
    t=(ExpirationDate-CurrentDate).days
    divTimes=(Dividend_date-CurrentDate).days

    s=151.03
    strike=data['Strike']
    call=True if data['OptionType']=='Call' else False
    rf=0.0425
    c=rf
    ttm=t/tradingDayYear
    P=data['CurrentPrice']

    N=200
    divTimes=[divTimes/t*N]
    divAmts=[1]

    def f(vol):
        return binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,call,divAmts,divTimes) - P
    iVol = optimize.root_scalar(f, bracket=[1e-6, 10], method='brentq')
    return iVol.root

iVol=df.apply(findImpliedVol_plus,axis=1)
df['Implied Volatility']=iVol

/var/folders/sg/h9ttt9dn0818s7d6p89mw_g00000gn/T/ipykernel_14724/1274203432.py:49: RuntimeWarning: overflow encountered in double_scalars
  optionValues[idx]=max(optionValues[idx],df*(pu*optionValues[idxFunction(i+1,j+1)] + pd*optionValues[idxFunction(i,j+1)]))
/var/folders/sg/h9ttt9dn0818s7d6p89mw_g00000gn/T/ipykernel_14724/1274203432.py:49: RuntimeWarning: invalid value encountered in double_scalars
  optionValues[idx]=max(optionValues[idx],df*(pu*optionValues[idxFunction(i+1,j+1)] + pd*optionValues[idxFunction(i,j+1)]))
/var/folders/sg/h9ttt9dn0818s7d6p89mw_g00000gn/T/ipykernel_14724/3122062256.py:59: RuntimeWarning: invalid value encountered in double_scalars
  optionValues[idx]=max(optionValues[idx],df*(pu*optionValues[idxFunction(i+1,j+1)] + pd*optionValues[idxFunction(i,j+1)]))


In [58]:
def binomial_tree_gbsm_american_div_df(data,prices,ndays=0):
    '''
    Designed for apply method of dataframe to accelarate handling speed.

    ndays - simulate price n days ahead
    prices - simulated price
    '''
    if data['Type']!='Option':
        return prices*data['Holding']

    strike=data['Strike']
    CurrentDate=datetime.datetime.strptime('03/03/2023','%m/%d/%Y')+datetime.timedelta(days=ndays)
    ExpirationDate=data['ExpirationDate']
    ExpirationDate=datetime.datetime.strptime(ExpirationDate,'%m/%d/%Y')
    Dividend_date=datetime.datetime.strptime('03/15/2023','%m/%d/%Y')
    t=(ExpirationDate-CurrentDate).days
    divTimes=(Dividend_date-CurrentDate).days
    tradingDayYear=365

    call=True if data['OptionType']=='Call' else False
    rf=0.0425
    c=rf
    ttm=t/tradingDayYear
    vol = data['Implied Volatility']
    N=20
    divTimes=[divTimes/t*N]
    divAmts=[1]

    positionValue=[]
    for s in prices:
        positionValue.append(binomial_tree_gbsm_american_div(s,strike,ttm,vol,rf,c,N,call,divAmts,divTimes))
    # list can not times -1, convert to array
    positionValue = np.array(positionValue)
    return positionValue * data['Holding']

In [510]:
# Fit the AR1 Model and simulate the return 10 days ahead 
priceData=pd.read_csv('DailyPrices.csv',index_col='Date')
logReturn=return_calculate(priceData.AAPL,option="CONTINUOUS",rm_means=True)
# fit Normal distribution
res=norm.fit(logReturn,floc=0)
norm.rvs(size=10,loc=res[0],scale=res[1])
# current Price
currentPrice=151.03
# simulate price ten days later
simulatedPrice=[]
N=100000 # number of simulations

simulatedReturns=[]
for _ in range(N):
    simulatedReturn=norm.rvs(size=10,loc=res[0],scale=res[1])
    simulatedReturns.append(simulatedReturn)
    tenDaysReturn=sum(simulatedReturn)
    simulatedPrice.append(currentPrice*np.exp(tenDaysReturn))
simulatedPrice=np.array(simulatedPrice)
simulatedReturns=np.array(simulatedReturns).flatten()

In [512]:
# apply those returns to the current AAPL price (above). Calculate Mean, VaR and ES
strategies=df.Portfolio.unique()
columns=['Mean(Portfolio Value)','Mean(Change)','VaR','ES']
ans=pd.DataFrame(index=strategies,columns=columns)
for idx,strategy in enumerate(strategies):
    data=df.query("Portfolio == @strategy")
    # Simulated Position Value
    positionValue=data.apply(binomial_tree_gbsm_american_div_df,prices=simulatedPrice,ndays=10,axis=1)
    # Initial Position Value
    iniValue=data.apply(binomial_tree_gbsm_american_div_df,prices=np.array([currentPrice]),axis=1)
    # Simulated Portfolio Value
    totalPortfolioValue=positionValue.sum()
    # Initial Portfolio Value
    iniValue=iniValue.sum()
    # Changed Portfolio Value
    valueChange=totalPortfolioValue-iniValue
    valueChange = np.array(valueChange)
    # calculate VaR and Expected Shortfall
    VaR_p = RiskMetrics.VaR_historical(valueChange,alpha=0.05)
    ES = RiskMetrics.ES_historical(valueChange,alpha=0.05)

    # insert Mean, VaR, ES to dateframe
    mean_var_es=[totalPortfolioValue.mean(),valueChange.mean(),VaR_p,ES]
    ans.loc[strategy]=mean_var_es

ans
    

,Mean(Portfolio Value),Mean(Change),VaR,ES
Straddle,13.573306,2.034698,1.175029,1.221071
SynLong,1.756892,-0.163474,17.931696,22.04644
CallSpread,4.503748,0.034049,3.760683,4.071643
PutSpread,3.426409,0.508251,2.670079,2.794192
Stock,151.392764,0.362764,16.740359,20.588025
Call,7.665099,0.935612,5.97523,6.309565
Put,5.908207,1.099086,4.495994,4.656884
CoveredCall,146.359195,-0.595879,12.87181,16.616972
ProtectedPut,155.276754,1.22444,7.485625,7.832187


# Problem 3
Use the Fama French 3 factor return time series (F-F_Research_Data_Factors_daily.CSV) as well as the Carhart Momentum time series (F-F_Momentum_Factor_daily.CSV) to fit a 4 factor model to the following stocks.

AAPL FB UNH MA

MSFT NVDA HD PFE

AMZN BRK-B PG XOM

TSLA JPM V DIS

GOOGL JNJ BAC CSCO

Fama stores values as percentages, you will need to divide by 100 (or multiply the stock returns by 100) to get like units.

Based on the past 10 years of factor returns, find the expected annual return of each stock.

Construct an annual covariance matrix for the 10 stocks.

Assume the risk free rate is 0.0425. Find the super efficient portfolio.

In [349]:
# parse the data of fama 3-factor
fama_data=pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
fama_data['Date']=fama_data['Date'].apply(str)
fama_data=fama_data.set_index(['Date'])
fama_data.index=pd.to_datetime(fama_data.index,format='%Y%m%d')
fama_data/=100
# parse the data of momentum
mom_data=pd.read_csv('F-F_Momentum_Factor_daily.CSV')
mom_data['Date']=mom_data['Date'].apply(str)
mom_data=mom_data.set_index(['Date'])
mom_data.index=pd.to_datetime(mom_data.index,format='%Y%m%d')
mom_data/=100
# combine them together
fama_data=pd.concat([fama_data,mom_data],axis=1)

# parse the price data 
priceData=pd.read_csv('DailyPrices.csv',index_col='Date')
priceData.index=pd.to_datetime(priceData.index)
# calculate the return
logReturn=return_calculate(priceData,option="CONTINUOUS",rm_means=True)
logReturn.index=pd.to_datetime(logReturn.index)

In [299]:
stock_list=['AAPL','META','UNH','MA',
           'MSFT','NVDA','HD','PFE',
           'AMZN','BRK-B','PG','XOM',
           'TSLA','JPM','V','DIS',
           'GOOGL','JNJ','BAC','CSCO']

In [354]:
# find the intersected date between fama data and return data
intersected_date=list(set(logReturn.index).intersection(set(fama_data.index )))
rt=logReturn.loc[intersected_date,:].sort_index(axis=0)
excess_r=rt.sub(fama_data.loc[intersected_date,'RF'].sort_index(axis=0),axis=0)
# get the return and excess return data of given stock list
rt=rt[stock_list]
excess_r=excess_r[stock_list]

# Ge the X to do the regression (3 factors)
x3=fama_data.loc[intersected_date,:'HML']
x3['intersect']=1
x3=x3.sort_index(axis=0)
# Calculate the beta
X3=np.array(x3)
beta3=linalg.inv(X3.T@X3)@X3.T@excess_r
beta3.index=x3.columns
beta3

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Mkt-RF,1.189278,1.298217,0.679940,1.083981,1.131225,1.851955,0.843703,0.637883,1.389100,0.924210,0.542710,0.999963,1.361001,1.081494,0.946758,1.107542,1.146552,0.371472,1.208477,0.888754
SMB,-0.603742,-0.038264,-0.777046,-0.131286,-0.834583,0.369798,0.176184,-0.620730,-0.419800,-0.169486,-0.634187,0.433576,0.100989,-0.164068,-0.288914,0.466203,-0.503428,-0.608642,-0.122281,-0.349312
HML,-0.382008,-0.771283,0.091228,0.051469,-0.564400,-0.589011,-0.073920,0.122969,-0.661401,0.438675,-0.037065,1.357141,-1.008977,0.554808,-0.034889,0.071042,-0.537634,-0.035647,0.659408,0.132487
intersect,0.000248,-0.000279,0.000333,0.000214,0.000256,-0.000068,0.000233,0.000233,0.000690,0.000125,0.000336,-0.000171,-0.000247,0.000057,0.000241,-0.000002,0.000583,0.000193,0.000014,0.000337


In [355]:
# Ge the X to do the regression (4 facotrs)
x4=pd.concat([fama_data.loc[intersected_date,:'HML'],fama_data.loc[intersected_date,'Mom']],axis=1)
x4['intersect']=1
x4=x4.sort_index(axis=0)
# Calculate the beta
X4=np.array(x4)
beta4=linalg.inv(X4.T@X4)@X4.T@excess_r
beta4.index=x4.columns
beta4

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Mkt-RF,1.217725,1.148571,0.806204,1.080449,1.144474,1.826236,0.826348,0.686452,1.272981,0.888726,0.610584,1.044213,1.322602,0.992086,0.941386,0.986327,1.085086,0.409829,1.108641,0.956303
SMB,-0.517700,-0.490887,-0.395143,-0.141970,-0.794509,0.292008,0.123690,-0.473826,-0.771015,-0.276811,-0.428894,0.567414,-0.015155,-0.434493,-0.305162,0.099573,-0.689337,-0.492627,-0.424248,-0.145002
HML,-0.468374,-0.316958,-0.292111,0.062193,-0.604625,-0.510929,-0.021229,-0.024487,-0.308865,0.546404,-0.243131,1.222800,-0.892396,0.826249,-0.018580,0.439051,-0.351026,-0.152098,0.962510,-0.072591
Mom,0.134111,-0.705488,0.595259,-0.016652,0.062463,-0.121248,-0.081820,0.228974,-0.547427,-0.167284,0.319984,0.208609,-0.181030,-0.421502,-0.025326,-0.571454,-0.289770,0.180829,-0.470666,0.318451
intersect,0.000247,-0.000270,0.000325,0.000214,0.000255,-0.000066,0.000234,0.000229,0.000698,0.000127,0.000331,-0.000174,-0.000245,0.000063,0.000241,0.000005,0.000587,0.000191,0.000020,0.000333


In [388]:
# past 10 years of factor returns
expected_fama_factor=fama_data.loc[fama_data.index[-1]-pd.DateOffset(years=10):,:].mean()
expected_fama_factor['intersect']=1

# past 10 years of risk-free rate
expected_RF=expected_fama_factor['RF']

# 3-factor returns
expected_fama_factor3 = expected_fama_factor[["Mkt-RF","SMB","HML","intersect"]]
# Use the fama Model to get the expected return (3-factor)
expected_return3=expected_fama_factor3 @ beta3 + expected_RF
expected_return3=expected_return3*252


# 4-factor returns
expected_fama_factor4 = expected_fama_factor[["Mkt-RF","SMB","HML","Mom","intersect"]]
# Use the fama Model to get the expected return (3-factor)
expected_return4=expected_fama_factor4 @ beta4 + expected_RF
expected_return4=expected_return4*252

# Covariance of returns
cov=rt.cov()*252
cov

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.127207,0.133582,0.041082,0.082998,0.103214,0.173362,0.067035,0.033613,0.124380,0.056524,0.037892,0.039289,0.159238,0.059442,0.073237,0.088992,0.109646,0.023176,0.066732,0.067206
META,0.133582,0.366743,0.031484,0.103247,0.132369,0.232108,0.090646,0.044961,0.180497,0.061023,0.035097,0.028479,0.169147,0.075850,0.087893,0.120721,0.162957,0.020405,0.087751,0.073321
UNH,0.041082,0.031484,0.059278,0.032487,0.040316,0.050483,0.029486,0.033386,0.041372,0.029396,0.028624,0.025609,0.042490,0.033932,0.030321,0.025629,0.036191,0.023928,0.036079,0.030867
MA,0.082998,0.103247,0.032487,0.098255,0.081182,0.139949,0.057923,0.034706,0.098644,0.048613,0.031772,0.031895,0.100754,0.059844,0.084645,0.079457,0.079355,0.017773,0.064977,0.052842
MSFT,0.103214,0.132369,0.040316,0.081182,0.126110,0.174274,0.070534,0.035745,0.131600,0.052929,0.035238,0.032994,0.132900,0.058029,0.069834,0.087380,0.116143,0.020215,0.065664,0.060984
NVDA,0.173362,0.232108,0.050483,0.139949,0.174274,0.403840,0.112599,0.048095,0.221372,0.085702,0.043357,0.057871,0.295550,0.101122,0.121207,0.159287,0.185502,0.021682,0.115146,0.099811
HD,0.067035,0.090646,0.029486,0.057923,0.070534,0.112599,0.097918,0.033878,0.094110,0.042989,0.035283,0.018066,0.079748,0.045224,0.051793,0.064000,0.066452,0.022486,0.047420,0.048605
PFE,0.033613,0.044961,0.033386,0.034706,0.035745,0.048095,0.033878,0.071918,0.037790,0.032105,0.028216,0.021071,0.023728,0.032781,0.031744,0.024953,0.029645,0.028077,0.031570,0.030058
AMZN,0.124380,0.180497,0.041372,0.098644,0.131600,0.221372,0.094110,0.037790,0.237443,0.066251,0.031645,0.041800,0.191454,0.074458,0.086871,0.123706,0.143582,0.022283,0.087440,0.070910
BRK-B,0.056524,0.061023,0.029396,0.048613,0.052929,0.085702,0.042989,0.032105,0.066251,0.050832,0.025543,0.034482,0.064069,0.048016,0.043147,0.052258,0.055117,0.019663,0.051790,0.040984


In [513]:
# expected return
expected_return4

AAPL     0.230566
META     0.076454
UNH      0.203863
MA       0.196950
MSFT     0.223876
NVDA     0.223627
HD       0.169202
PFE      0.156325
AMZN     0.338920
BRK-B    0.143935
PG       0.175961
XOM      0.086125
TSLA     0.118198
JPM      0.134090
V        0.186975
DIS      0.119381
GOOGL    0.291816
JNJ      0.112092
BAC      0.135887
CSCO     0.217847
dtype: float64

In [505]:
def super_efficient_portfolio(expected_rts,cov):
    '''Given a target return, use assets to find the optimal portfolio with lowest risk'''
    rf=0.0425
    fun=lambda wts: -(wts@expected_rts-rf)/np.sqrt(wts@cov@wts)
    x0 = np.full(expected_rts.shape[0],1/expected_rts.shape[0])
    cons = [{'type':'ineq', 'fun':lambda x:x},
        {'type':'eq', 'fun':lambda x:sum(x)-1}]
    bounds = [(0, 1) for _ in range(expected_rts.shape[0])]
    res = optimize.minimize(fun, x0, method='SLSQP',bounds=bounds,constraints=cons)
    return res
        

In [502]:
# 3-factor's super efficient portfolio
x3=super_efficient_portfolio(expected_return3,cov).x
pd.DataFrame(x3,index=stock_list,columns=['Weight']).round(2).T

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Weight,0.0,0.0,0.26,0.0,0.0,0.0,0.0,0.0,0.17,0.0,0.31,0.0,0.0,0.0,0.0,0.0,0.23,0.0,0.0,0.03


In [504]:
# 4-factor's super efficient portfolio
x4=super_efficient_portfolio(expected_return4,cov).x
pd.DataFrame(x4,index=stock_list,columns=['Weight']).round(2).T

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Weight,0.0,0.0,0.37,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.28,0.0,0.0,0.0,0.0,0.0,0.17,0.0,0.0,0.08


In [514]:
# 4-factor's super efficient portfolio sharpe
-super_efficient_portfolio(expected_return4,cov).fun

0.832215456951802